In [ ]:
library(rEDM)

#loading
dailyC <- read.csv("TransmissionDaily.csv")


In [ ]:
#check cols name
str(dailyC)


In [ ]:
#showing original graph

# Create a new data frame with time and colonized count
df <- data.frame(
  time = dailyC $Time,
  dailyCount = dailyC $DailyCount
)

# Plot the colonized counts over time
plot(df$time, df$dailyCount, type = "l",
     xlab = "Time", ylab = "Daily Detected Patients",
     lwd = 2)


In [ ]:
#Finding E by Optimal embedding dimension 
#only need minimum dimension

E.opt <- EmbedDimension(dataFrame = dailyC,
                       lib = "1 170", 
                       pred = "171 355",
                       columns = "DailyCount",
                       target = "DailyCount")


In [ ]:
#Predict Interval(replace the E with detected E at peak)
#run Simplex

simplex_out <- Simplex(dataFrame = dailyC,
                       lib = "1 170", # training range
                       pred = "171 355",# prediction range
                       columns = "DailyCount",
                       target = "DailyCount",
                       E = 4)
simplex_out[c(1:2, 300:301), ] 


In [ ]:
# Plot actual vs predicted colonized counts
plot(dailyC$Time, dailyC$DailyCount, type = "l", lwd = 2,
     xlab = "Time", ylab = "Daily Detected Patients")

lines(simplex_out$Time, simplex_out$Predictions, col = "red", lwd = 2)

legend("topleft", legend = c("Observed", "Predicted (t + 1)"),
       fill = c("black", "red"), bty = "n", cex = 1.3)

# type “l” means line, lwd =2 means line width
# observed in black, predicted in red
#bty means no box around legend, cex means font size


In [ ]:
#Evaluate performance
ComputeError(simplex_out$Observations, simplex_out$Predictions)

In [ ]:
# prediction Decay(for short term is good, long term has decay in accuracy)
# Tp increases, p decreases as prediction decay, p drops to 0, then chaotic
# normal in nonlinear system
rho_Tp<- PredictInterval(dataFrame = dailyC, 
                       lib = "1 170", 
                       pred = "171 355",
                         columns = "DailyCount",
                         target = "DailyCount",
E = 4) 

In [ ]:
#Test for nonlinearity(Smap)
# If ρ increases as θ increases, the system likely has nonlinear behavior
# If ρ is highest at θ = 0, it's likely a linear or stochastic process

rho_theta  <- PredictNonlinear(dataFrame = dailyC,
                       lib = "1 170", 
                       pred = "171 355",
                         columns = "DailyCount",
                         target = "DailyCount",
                              E = 4)

#Likely a linear or stochastic process


In [ ]:
#For Smap:

Smap  <- SMap(dataFrame = dailyC,
                       lib = "1 170", 
                       pred = "171 355",
                         columns = "DailyCount",
                         target = "DailyCount",
                    E = 4,
                    theta = 3)

head(cbind(Smap$predictions, Smap$coefficients), 3)

tail(cbind(Smap$predictions, Smap$coefficients), 3) 
